https://www.nytimes.com/games/wordle/index.html

In [53]:
import re
import string
import pandas as pd
from urllib.request import urlopen

In [54]:
def vowel_count(word):
    vwels = "AEIOU"
    st = set(word.upper())
    vc = 0
    for v in vwels:
        vc += v in st
    return vc

In [55]:
def scrabble_score(word):
    scrob_d = {'A': 1, 'B': 3, 'C': 3,  'D': 2, 'E': 1, 'F': 4, 'G': 2,
               'H': 4, 'I': 1, 'J': 8,  'K': 5, 'L': 1, 'M': 3, 'N': 1,
               'O': 1, 'P': 3, 'Q': 10, 'R': 1, 'S': 1, 'T': 1, 'U': 1,
               'V': 4, 'W': 4, 'X': 8,  'Y': 4, 'Z': 10}

    uword = word.upper()
    sc = 0
    for scrab_letter in uword:
        sc += scrob_d.get(scrab_letter, 0)
    return sc

In [56]:
def unique_letters2(word, letter_set1, letter_set2):
    x = set(word.upper())
    y = set(letter_set1)
    z = set(letter_set2)
    q = x.difference(y).difference(z)
    return len(q)

In [57]:
class Wordle:

    def __init__(self, omt, iclude, akey):
        self.word_length = 5
        self.u = 'https://norvig.com/ngrams/enable1.txt'

        self.f = urlopen(self.u)
        self.w = self.f.read().decode('utf-8').upper().split()
        self.w = list(map(lambda word: word.upper(), self.w))
        self.w = list(map(lambda word: word.translate(
                  str.maketrans('', '', string.punctuation)), self.w))
        self.w = list(filter(lambda word: len(word) == self.word_length, self.w))

        self.a = ''.join(akey.split())
        self.a = self.a[:self.word_length].upper()
        self.a = re.sub("[^A-Z]", "?", self.a)

        self.o = omt.upper()
        self.o = re.sub("[^A-Z]", "", self.o)

        self.i = iclude.upper()
        self.i = re.sub("[^A-Z]", "", self.i)

        self.nlc = []

        headers = ['Word', 'New Lttrs', 'Vowels', 'Score']
        self.df = pd.DataFrame(columns=headers)

        self.rws = 0

    def filt(self):
        for lttr in self.o:
            self.w = [word for word in self.w if lttr not in word]

        for lttr in self.i:
            self.w = [word for word in self.w if lttr in word]

        if self.a != '?????':
            patt = self.a.replace("?", "[A-Z]{1}")
            r = re.compile(patt)
            self.w = list(filter(r.match, self.w))

    def new_letters(self):
        y = set(self.a)
        z = set(self.i)
        for word in self.w:
            x = set(word)
            q = x.difference(y).difference(z)
            self.nlc.append(len(q))

    def build_df(self):
        vowels = list(map(vowel_count, self.w))
        scrab = list(map(scrabble_score, self.w))
        df_raw = {'Word': self.w, 'New Lttrs': self.nlc, 'Vowels': vowels, 'Score': scrab}
        temp = pd.DataFrame(df_raw)
        self.df = pd.concat([self.df, temp])
        self.df = self.df.sort_values(['New Lttrs', 'Vowels', 'Score', 'Word'],
                                      ascending=(False, False, True, False))
        self.rws = len(self.df)
        pd.set_option('display.max_rows', self.rws)
        display(self.df)

In [71]:
omit = 'rise mou gy'  # These are the Black Letters
include = 'a nt'  # These are the Yellow Letters
answr = ' ? a ? ? ? '  # These are the Green Letters

In [72]:
oW = Wordle(omit, include, answr)

In [73]:
oW.filt()
oW.new_letters()
oW.build_df()

,Word,New Lttrs,Vowels,Score
1,NATCH,2,1,10
0,NATAL,1,1,5
3,TANKA,1,1,9
2,QANAT,1,1,14
